# Kaggle Data into Elasticsearch

In [1]:
import os, glob
import pandas as pd
import numpy as np

In [5]:
kgdownload_loc = "/Users/kaushalk/Downloads/covid19-in-india/"
kgfiles = os.listdir(kgdownload_loc)

In [6]:
def load_kgdata(loc, file):
    df1 = pd.read_csv((loc+ file[1]))
    df2 = pd.read_csv(loc+ file[2])
    df3 = pd.read_csv(loc+ file[3])
    df4 = pd.read_csv(loc+ file[4])
    df5 = pd.read_csv(loc+ file[5])
    return df1,df2,df3,df4,df5

icmr_test, hosp_beds, age_grp, pop_ind, indv_df = load_kgdata(kgdownload_loc,kgfiles) 

In [7]:
indv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1304 entries, 0 to 1303
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1304 non-null   int64 
 1   government_id       257 non-null    object
 2   diagnosed_date      1304 non-null   object
 3   age                 485 non-null    object
 4   gender              533 non-null    object
 5   detected_city       643 non-null    object
 6   detected_district   955 non-null    object
 7   detected_state      1085 non-null   object
 8   nationality         414 non-null    object
 9   current_status      1085 non-null   object
 10  status_change_date  1082 non-null   object
 11  notes               1060 non-null   object
dtypes: int64(1), object(11)
memory usage: 122.4+ KB


In [8]:
indv_df = indv_df[indv_df.count(1) > 4] # Keep only reocrds where more than half of the column reocrds not null
indv_df.nationality.replace({"Indian": "India"}, inplace=True) # Replace Indian with India 
indv_df.nationality.fillna("India",inplace=True) # fill null value with India assuming all are indian patient 
indv_df.notes.fillna('NotAvl', inplace=True) # Fill null value in notes with "unknown" status
indv_df.status_change_date.fillna(indv_df.diagnosed_date,inplace=True) # replace status date nan value with dignosed date
indv_df[['government_id','gender', 'detected_city', 'detected_district']] = indv_df[['government_id','gender', 
                                                                                     'detected_city', 'detected_district']].fillna('NotAvl')

In [9]:
indv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1085 entries, 0 to 1084
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1085 non-null   int64 
 1   government_id       1085 non-null   object
 2   diagnosed_date      1085 non-null   object
 3   age                 485 non-null    object
 4   gender              1085 non-null   object
 5   detected_city       1085 non-null   object
 6   detected_district   1085 non-null   object
 7   detected_state      1085 non-null   object
 8   nationality         1085 non-null   object
 9   current_status      1085 non-null   object
 10  status_change_date  1085 non-null   object
 11  notes               1085 non-null   object
dtypes: int64(1), object(11)
memory usage: 110.2+ KB


In [10]:
indv_df.age.replace({'28-35':'32'}, inplace=True) # Fill Age 
indv_df['age'] = indv_df.age.fillna(-1).astype(int)

In [11]:
indv_df['diagnosed_date'] = pd.to_datetime(indv_df['diagnosed_date']).dt.date
indv_df['status_change_date'] = pd.to_datetime(indv_df['status_change_date']).dt.date

In [12]:
indv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1085 entries, 0 to 1084
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  1085 non-null   int64 
 1   government_id       1085 non-null   object
 2   diagnosed_date      1085 non-null   object
 3   age                 1085 non-null   int64 
 4   gender              1085 non-null   object
 5   detected_city       1085 non-null   object
 6   detected_district   1085 non-null   object
 7   detected_state      1085 non-null   object
 8   nationality         1085 non-null   object
 9   current_status      1085 non-null   object
 10  status_change_date  1085 non-null   object
 11  notes               1085 non-null   object
dtypes: int64(2), object(10)
memory usage: 110.2+ KB


In [13]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk 

In [15]:
es = Elasticsearch(['http://localhost:9200'], 
                    http_auth=('user', 'passXXXX'), timeout = 3000)
    # delete index if exists
if es.indices.exists('covid19india-indvdb'):
    es.indices.delete(index='covid19india-indvdb')


    
# Load the json doc in index
es.indices.create(index='covid19india-indvdb', ignore=400, body={})
documents = indv_df.to_dict(orient='records')
bulk(es, documents, index = 'covid19india-indvdb', raise_on_error = True)

(1085, [])

## CovidOrg data

In [78]:
data_url = "https://raw.githubusercontent.com/covid19india/CovidCrowd/master/data/raw_data.csv"
data_dir ="/Volumes/Lab/PROJECTS/COVID-19-AnalyticsHub/India/Data/"

df = pd.read_csv(data_url)


In [79]:
df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Estimated Onset Date', 'Age Bracket', 'Gender', 'Detected City',
       'Detected District', 'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes'],
      dtype='object')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 18 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Patient Number                             1053 non-null   float64
 1   State Patient Number                       245 non-null    object 
 2   Date Announced                             1053 non-null   object 
 3   Estimated Onset Date                       0 non-null      float64
 4   Age Bracket                                473 non-null    float64
 5   Gender                                     521 non-null    object 
 6   Detected City                              638 non-null    object 
 7   Detected District                          927 non-null    object 
 8   Detected State                             1053 non-null   object 
 9   Current Status                             1053 non-null   object 
 10  Notes                   

In [81]:
indv_df.columns

Index(['id', 'government_id', 'diagnosed_date', 'age', 'gender',
       'detected_city', 'detected_district', 'detected_state', 'nationality',
       'current_status', 'status_change_date', 'notes'],
      dtype='object')

In [82]:
df = df.drop(['Estimated Onset Date', 'Source_1', 'Source_2', 'Source_3', 'Backup Notes'], axis=1)
df.columns = ['id', 'government_id', 'diagnosed_date', 'age', 'gender',
       'detected_city', 'detected_district', 'detected_state', 'current_status', 
              'notes', 'contracted','nationality', 'status_change_date']
df.head()

,id,government_id,diagnosed_date,age,gender,detected_city,detected_district,detected_state,current_status,notes,contracted,nationality,status_change_date
0,1.0,KL-TS-P1,30/01/2020,20.0,F,Thrissur,Thrissur,Kerala,Recovered,Travelled from Wuhan,NaN,India,14/02/2020
1,2.0,KL-AL-P1,02/02/2020,NaN,NaN,Alappuzha,Alappuzha,Kerala,Recovered,Travelled from Wuhan,NaN,India,14/02/2020
2,3.0,KL-KS-P1,03/02/2020,NaN,NaN,Kasaragod,Kasaragod,Kerala,Recovered,Travelled from Wuhan,NaN,India,14/02/2020
3,4.0,DL-P1,02/03/2020,45.0,M,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,"Travelled from Austria, Italy",NaN,India,15/03/2020
4,5.0,TS-P1,02/03/2020,24.0,M,Hyderabad,Hyderabad,Telangana,Recovered,NaN,NaN,India,02/03/2020


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1053 non-null   float64
 1   government_id       245 non-null    object 
 2   diagnosed_date      1053 non-null   object 
 3   age                 473 non-null    float64
 4   gender              521 non-null    object 
 5   detected_city       638 non-null    object 
 6   detected_district   927 non-null    object 
 7   detected_state      1053 non-null   object 
 8   current_status      1053 non-null   object 
 9   notes               1047 non-null   object 
 10  contracted          243 non-null    object 
 11  nationality         406 non-null    object 
 12  status_change_date  1055 non-null   object 
dtypes: float64(2), object(11)
memory usage: 107.6+ KB


In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1053 non-null   float64
 1   government_id       245 non-null    object 
 2   diagnosed_date      1053 non-null   object 
 3   age                 473 non-null    float64
 4   gender              521 non-null    object 
 5   detected_city       638 non-null    object 
 6   detected_district   927 non-null    object 
 7   detected_state      1053 non-null   object 
 8   current_status      1053 non-null   object 
 9   notes               1047 non-null   object 
 10  contracted          243 non-null    object 
 11  nationality         406 non-null    object 
 12  status_change_date  1055 non-null   object 
dtypes: float64(2), object(11)
memory usage: 107.6+ KB


In [89]:
#str_ = !pwd
df = df[df.count(1) > 5]
df.nationality.fillna("India",inplace=True)
df.notes.fillna("no notest", inplace=True)
df['gender'] = df.gender.replace({"M": "Male","F": "Female"}).fillna("Not Available")
df['age'] = df.age.fillna(-1)
df.status_change_date.fillna(df.diagnosed_date,inplace=True) # replace status date nan value with dignosed date
df[['government_id', 'detected_city','detected_district', 'contracted']] = df[['government_id', 
                                                'detected_city', 'detected_district', 'contracted']].fillna('Not Available')
df['diagnosed_date'] = pd.to_datetime(df['diagnosed_date']).dt.date
df['status_change_date'] = pd.to_datetime(df['status_change_date']).dt.date
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1052 entries, 0 to 1052
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1052 non-null   float64
 1   government_id       1052 non-null   object 
 2   diagnosed_date      1052 non-null   object 
 3   age                 1052 non-null   float64
 4   gender              1052 non-null   object 
 5   detected_city       1052 non-null   object 
 6   detected_district   1052 non-null   object 
 7   detected_state      1052 non-null   object 
 8   current_status      1052 non-null   object 
 9   notes               1052 non-null   object 
 10  contracted          1052 non-null   object 
 11  nationality         1052 non-null   object 
 12  status_change_date  1052 non-null   object 
dtypes: float64(2), object(11)
memory usage: 115.1+ KB


In [90]:
df.head()

,id,government_id,diagnosed_date,age,gender,detected_city,detected_district,detected_state,current_status,notes,contracted,nationality,status_change_date
0,1.0,KL-TS-P1,2020-01-30,20.0,Female,Thrissur,Thrissur,Kerala,Recovered,Travelled from Wuhan,Not Available,India,2020-02-14
1,2.0,KL-AL-P1,2020-02-02,-1.0,Not Available,Alappuzha,Alappuzha,Kerala,Recovered,Travelled from Wuhan,Not Available,India,2020-02-14
2,3.0,KL-KS-P1,2020-03-02,-1.0,Not Available,Kasaragod,Kasaragod,Kerala,Recovered,Travelled from Wuhan,Not Available,India,2020-02-14
3,4.0,DL-P1,2020-02-03,45.0,Male,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,"Travelled from Austria, Italy",Not Available,India,2020-03-15
4,5.0,TS-P1,2020-02-03,24.0,Male,Hyderabad,Hyderabad,Telangana,Recovered,no notest,Not Available,India,2020-02-03


## Export into Elasticsearch Index

In [100]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk 

In [99]:
es = Elasticsearch(['http://localhost:9200'], 
                    http_auth=('user', 'passXXXX'), timeout = 3000)
    # delete index if exists
if es.indices.exists('covid19indiaorg'):
    es.indices.delete(index='covid19indiaorg')


    
# Load the json doc in index
es.indices.create(index='covid19indiaorg', ignore=400, body={})
documents = df.to_dict(orient='records')
bulk(es, documents, index = 'covid19indiaorg', raise_on_error = True)

(1052, [])

In [134]:
print(f'No of inserted records:', len(documents))

No of inserted records: 1052
